# TensorFlow Northwest



## Tensorflow Sign Language Code Along: DNN, Dropout, Hyperparameters



This Code Along will introduces using Tensorflow for image detection. In this code along, we will:

- Load images and convert them to numpy arrays using PIL
- Design Deep Neural Network (DNN) with a dropout layer.
- Run DNN with different hyperparameter settings


#### Import Tensorflow

In the first input cell, we will import tensorflow and verify version used.

Python execution of the cell should complete without any errors. You may get deprecated warnings, just ignore them. Let me try it below:


In [ ]:
import tensorflow as tf
print( tf.__version__ )

We will need a few more libraries, so let's load them now.

In [ ]:
import os
import time
from PIL import Image
import zipfile
import numpy as np

## Download Dataset

In [ ]:
if not os.path.isdir('gestures'):
    ! wget https://github.com/tensorflow-northwest/sign-lang/raw/master/gestures.zip
    with zipfile.ZipFile("gestures.zip","r") as zip_ref:
        zip_ref.extractall(".")
    ! rm gestures.zip

## Data Source

For each gesture, our dataset includes 1200 x 50x50pixel images in grayscale, stored in the gestures/ folder. The gestures/0/ folder contains 1200 blank images which signify "none" gesture.

Let's go to the downloaded images and see what's there. Let's first check that we are in the correct directory.

In [ ]:
! pwd
! ls -l

You will see under the gestures directory, we have folders 0 through 26. Folder 0 is the none image, while folders 1 through 26 have images for the letters A..Z, respectively.

Let's look under one of the folders (i.e., folder 1 for the letter A):

<img src='gestures.jpg'/>

Let's now read in a JPEG image and convert it to an uncompressed numpy 2D matrix of pixels.

In [ ]:
image = np.asarray(Image.open("gestures/5/9.jpg"))

# NOTE: to save-
# im = Image.fromarray(numpy.uint8(image))
# im.save("image.png")

Let's verify that the image is a numpy matrix as expected. Its type should be numpy.ndarray and it's shape should be 50 x 50 pixels.

In [ ]:
print( type(image) )
print( image.shape )

Let's see how the image looks like in an array. As you can see, it is a matrix of values between 0 and 255 (uint8).

In [ ]:
image

### Plotting an Image

Let's plot one of the images in the training set. To do so, we will use the plotting functions of matplotlib. Let's start by importing the package.

In [ ]:
import matplotlib.pyplot as plt

# This line is specific to python notebooks (not python). 
# It causes plots to automatically be rendered (displayed) without issuing a show command.
%matplotlib inline  

In [ ]:
# Let's plot it now
plt.imshow( image )

### Loading the Dataset

The function below traverses the gestures folder for each subfolder (letter in the alphabet) and loads the images and corresponding label (letter), which is obtained from the subfolder name.

In [ ]:
def load_dataset( verbose = False ):
    """ Load the Sign Language dataset for the Alphabet """
    images = []
    labels = []
    # Process each subfolder (0..26) in the gestures folder
    for subfolder in os.listdir("gestures"):
        if verbose == True: print("Loading Subfolder", subfolder)
        # There are 1200 images per letter
        for i in range(1200):
            # Read each image in
            # image = cv2.imread("gestures/"+ subfolder + "/" + str(i+1) + ".jpg", 0)
            image = np.asarray(Image.open("gestures/"+ subfolder + "/" + str(i+1) + ".jpg"))
            # if bad image, skip
            if np.any(image == None):
                continue
            # add image to images list
            images.append( image )
            # add corresponding label to labels list
            labels.append( subfolder )
    # return the list of images and corresponding labels
    return images, labels

Let's now load the images. We will set the parameter verbose to True to see the progress since this will take awhile.

In [ ]:
start = time.time()
images, labels = load_dataset( True )
print("Time to Load Images: "+ str(1000*(time.time() - start))+" ms")

Let's see we got what we expected. There are 27 folders (26 letters and None) with 1200 images each. We should have a list of 32,400 (27 * 1200) numpy 2D matrixes (aka the images).

In [ ]:
print( len(images) )

Let's now verify that the images (and labels) we read in by plotting one of the images and it's corresponding label.

In [ ]:
# Let's plot it now
index = 2407
plt.imshow( images[index] )
print ("y = " + str(labels[index]))

### Flatten the Image Data

Now we will flatten the data (reshape so all rows follow each other sequential in a single vector).

In [ ]:
# Convert python list to numpy array
images_np = np.asarray(images)
print( "Images as Numpy Array:", images_np.shape )

images_flatten = images_np.reshape(images_np.shape[0], -1)
print( "Images Flatten:", images_flatten.shape )


### Normalize the Image Data

Now we will normalize the pixel values between 0 and 1.

In [ ]:
# Normalize the Pixel Values between 0 and 1
x = images_flatten / 255.0

### Categorical Conversion of Labels

Let's now convert the labels from a categorical value to a one hot encoding.

In [ ]:
# Convert python list to numpy array
labels_np = np.asarray(labels)
print( "Labels as Numpy Array:", labels_np.shape )

# Let's look at the array data type
labels_np

That's an unsigned 64bit integer. For our purposes we need it as a signed integer. Let's change its datatype.

In [ ]:
labels_int = labels_np.astype(np.int64)

# Let's look at the new data type
labels_int

Now the array shape and data type are ready. Let's do the one hot encoding.

In [ ]:
def convert_labels_to_one_hot_encoding(Y, C):
    """ This function will do the reshape and conversion (from Coursera)"""
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

# Let's do the conversion
y = convert_labels_to_one_hot_encoding(labels_int, 27)

# Let's print the shape of our labels
print( y.shape )

### Split Dataset into Training and Test Data

Now that we have our data and labels ready, let's split our dataset into training and test data. For our purposes, we will use 80% as training and 20% as test. We could do the split by hand using numpy.random.shuffle() to randomize the order, and then use array slicing.

But scikit-learn has an all-in-one method for this purpose, so we will use it.

In [ ]:
# Split arrays or matrices into random train and test subsets
from sklearn.model_selection import train_test_split

# Split our data/labels into 80% training and 20% test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

## Deep Neural Network with Dropout Layer - Softmax Activation

We are going to build a simple deep neural network (DNN) with a dropout layer. There will not be a convolutional layer. That is, we will use all 2500 pixels as input. The output layer from our neural network will be passed through a softmax activation function to produce our predictions of the letter.

In our Neural Network, we will have the following:

    - An input layer of 2500 inputs and 64 outputs
    - A linear recitifier activation function
    - A first hidden layer of 64 inputs and 32 outputs
    - A dropout layer
    - A linear recitifier activation function
    - A second hidden layer of 32 inputs and 20 outputs
    - A linear recitifier activation function
    - An output layer of 20 inputs and 27 outputs
    - A softmax activation function

INPUT LAYER => RELU => HIDDEN LAYER => DROPOUT LAYER => RELU => HIDDEN LAYER => RELU => OUTPUT LAYER => SOFTMAX

### Softmax

We will use the softmax layer to make our predictions. Each output from softmax will be a number between 0 and 1, representing a percent. That is, if the output for the node 3 is 0.8, then this means 80% prediction. We will choose the output with the highest percent when making a prediction.

Softmax is a mathematical function that takes a set of values, which may otherwise not add up to 1, and outputs a new set of numbers when totaled will add up to 1. That is, we use softmax() so that all our outputs for each image add up to 1 (100%).

Softmax will be our 'activation' function from the output layer.

## Building your first neural network in TensorFlow

### Design, then Run

#### Design

     -- Create placeholders for your input data
     -- Design the layers as a Graph
     -- Set the optimizer
     
#### Run

      -- Initialize the Graph
      -- Set number of epochs
      -- Set batch size, learning rate
      -- Run the Graph with the Training Data to Train a Model
      -- Validate the Model with Test Data


### Input Vector and Output Vector and Hyperparameter Placeholders

For our first tensorflow step, we will setup Tensorflow placeholders.

We have two placeholders we need to declare, one for the input vector (pixel image data) and one for the output vector (letter classifier).

For our input placeholder (which we call X), we have 2,500 features (pixels per image). For the output vector (which we call Y), we have have 27 classifiers (alphabet and None). In both cases, we set the second dimension of our vector to None. The None is a placeholder for the number of samples we will feed into the neural network. We also know that our data is floating point values between 0 and 1, so we will set the data type to float32.

We will declare two more placeholders for setting some hyper-parameters, the percent to keep in the dropout layer (D, and the learning rate in the optimizer (L). Since both are scalar values, we will define their shape as a single value.

In [ ]:
# Let's first reset our graph, so our neural network components are all declared within the same graph
from tensorflow.python.framework import ops
ops.reset_default_graph() 

In [ ]:
X = tf.placeholder(tf.float32, shape=[2500, None])
Y = tf.placeholder(tf.float32, shape=[27, None])
D = tf.placeholder(tf.float32, [])
L = tf.placeholder(tf.float32, [])

### INPUT LAYER

Let's now design our input layer. We need two things: weights and biases. 

Each input feature (pixel) will need a weight (which our model will learning during training). The weight is multipled against the value of the input (pixel), which we symbolically represent as Wx. 

Each output from the layer will need a bias (which our model will learning during training). The bias is added to the result of the weight multipled by the pixel value (Wx).

Let's create two Tensorflow variables for our weights and biases. The weights (which we call W) will need to be a 2D matrix. The rows are the number of inputs, which is 2500 and the columns the number of outputs to the hidden layer, which will be 64.

The bias will be a vector of size 64 (one for each output).

We need to initialize our weights and biases to some initial value. We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
tf.set_random_seed(1)   # Set the same seed to get the same initialization as in this demo.

# The weights for the input layer
W1 = tf.get_variable("W1", [64, 2500], initializer=tf.contrib.layers.xavier_initializer(seed=1))

# The bias for the output from the input layer
b1 = tf.get_variable("b1", [64, 1], initializer=tf.zeros_initializer())

Let's put it together into an input layer. We will use the Tensorflow method tf.matmul() to do a matrix multiplication of the weights (our variable W1) and the inputs (our placeholder X), add in the bias (b1), and pass the output through a linear activation function.

- Create a node that will multiply the weights (W1) against the input vector (X - which is our input placeholder).
- Create a node that adds the bias to the above node (W1 * X)
- Pass the outputs from the input layer through a RELU activation function

In [ ]:
# The first layer (input layer)
Z1 = tf.add(tf.matmul(W1, X), b1)

# Let's add the activation function to the output signal from the first layer
A1 = tf.nn.relu(Z1)

### FIRST HIDDEN LAYER

The first hidden layer will have 64 inputs (outputs from input layer) and 32 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W2 = tf.get_variable("W2", [32, 64], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b2 = tf.get_variable("b2", [32, 1], initializer=tf.zeros_initializer())

Let's construct the first hidden layer

- Create a node that will multiply the weights (W2) against the outputs of the input layer (A1).
- Create a node that adds the bias to the above node (W2 * A1)
- Pass the outputs from the (first) hidden layer through a dropout layer
- Pass the outputs from the dropout layer through a RELU activation function

In [ ]:
# The second layer (first hidden layer)
Z2 = tf.add(tf.matmul(W2, A1), b2) 

# Let's add the dropout layer to the output signal from the second layer
D2 = tf.nn.dropout(Z2, keep_prob=D)

# Let's add the activation function to the output signal from the dropout layer
A2 = tf.nn.relu(D2)

### SECOND HIDDEN LAYER

The second hidden layer will have 32 inputs (outputs from first hidden layer) and 20 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W3 = tf.get_variable("W3", [20, 32], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b3 = tf.get_variable("b3", [20, 1], initializer=tf.zeros_initializer())

Let's construct the second hidden layer.
- Create a node that will multiply the weights (W3) against the outputs of the first hidden layer (A2).
- Create a node that adds the bias to the above node (W3 * A2)
- Pass the outputs from the second hidden layer through a RELU activation function

In [ ]:
# The third layer (second hidden layer)
Z3 = tf.add(tf.matmul(W3, A2), b3) 

# Let's add the activation function to the output signal from the third layer
A3 = tf.nn.relu(Z3)

### OUTPUT LAYER

The output layer will have 20 inputs (outputs from the second hidden layer) and 27 outputs (one for each letter and None). Each input will need a weight and each output a bias (which we will train). The 27 outputs will be passed through a softmax activation function. 

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W4 = tf.get_variable("W4", [27, 20], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b4 = tf.get_variable("b4", [27, 1], initializer=tf.zeros_initializer())

Let's construct the output layer.

- Create a node that will multiply the weights (W4) against the outputs of the second hidden layer (A3).
- Create a node that adds the bias to the above node (W4 * A3)
- Pass the outputs from the output layer through a SOFTMAX squashing function (done by the optimizer)

In [ ]:
# The fourth layer (output layer)
Z4 = tf.add(tf.matmul(W4, A3), b4) 

## OPTIMIZER

Now its time to design our optimizer. Let's start by designing our cost function. We will use the mean value of the softmax cross entropy between the predicted labels and actual labels. This is what we want to reduce on each batch (aka the cost).

In [ ]:
# to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(Z4), labels=tf.transpose(Y)))

Let's design our optimizer. This is the method that adjusts the values of the weights and biases, based on minizing the cost value during training.

We also need to set a learning rate. This is multiplied against the gradient calculation. It's used to prevent huge swings in setting weights which can result in either converging at a local (instead of global) optima, or not converging at all (infinite gradient). We will set the learning rate when we run the graph using the placeholder L.

In [ ]:
# The learning rate for Gradient Descent algorithm

optimizer = tf.train.GradientDescentOptimizer(L).minimize(cost)

### Run the Graph

We've built our Tensorflow graph for training our data. So, let's start training it.

First, we need to call Tensorflow's global_variables_initializer() method to initialize the variables we've defined. We will create this as another node, which will be the first node we run (evaluate) in our graph.

In [ ]:
init = tf.global_variables_initializer()

#### Let's set our hyperparameters.

We need to set the number of epochs (that's how many times we run the training data through the neural network), and the batch size. The batch size is a small subset of the entire training set. We will be running a batch at a time per epoch. After each batch, then the cost is computed and backpropagated through the neural network.

In [ ]:
epochs = 20                                    # run a 20 epochs

batch_size = 100                               # for each epoch, train in batches of 100 images

number_of_images = x_train.shape[0]            # number of images in training data

batch_size = 200                               # size of each batch

# Feed Dictionary Parameters

keep_prob = 0.9                                # percent of outputs to keep in dropout layer

learning_rate = 0.5                            # the learning rate for graident descent

We are going to run the graph now!

We start by creating a tensorflow session (tf.Session()). Within the session we can run (evaluate) parts of the graph we designed.

We start by initializing the tensor variables we defined for the weights and biases.

We then run our training data through our neural network for the number of epochs we defined. For each epoch, we get a randomly shuffled batch from the training data and feed the batch (i.e. feed dictionary) into the neural network by running (evaluate) the optimizer node in our graph.

Once we've trained the model, then we create some new nodes to calculate accuracy and evaluate against the training data.

In [ ]:
def train():
    start = time.time()

    with tf.Session() as sess:
        # Initialize the variables
        sess.run(init)
        
        # number of batches in an epoch
        batches = number_of_images // batch_size

        # run our training data through the neural network for each epoch
        for epoch in range(epochs):

          epoch_cost = 0

          # Run the training data through the neural network
          for batch in range(batches):

              # Calculate the start and end indices for the next batch
              begin = (batch * batch_size)
              end   = (batch * batch_size) + batch_size

              # Get the next sequential batch from the training data
              batch_xs, batch_ys = x_train[begin:end], y_train[begin:end]

              # Feed this batch through the neural network.
              _, batch_cost = sess.run([optimizer, cost], feed_dict={X: batch_xs.T, Y: batch_ys.T, D: keep_prob, L: learning_rate})

              epoch_cost += batch_cost

          print("Epoch: ", epoch, "   Cost: ", epoch_cost / batches)

        print("Training Time: "+ str(1000*(time.time() - start))+" ms")
        # Test the Model

        # Let's select the highest percent from the softmax output per image as the prediction.
        prediction = tf.equal(tf.argmax(Z4), tf.argmax(Y))

        # Let's create another node for calculating the accuracy
        accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

        # Now let's run our trainingt images through the model to calculate our accuracy during training
        # Note how we set the keep percent for the dropout rate to 1.0 (no dropout) when we are evaluating the accuracy.
        print ("Train Accuracy:", accuracy.eval({X: x_train.T, Y: y_train.T, D: 1.0}))

        # Now let's run our test images through the model to calculate our accuracy on the test data
        print ("Test Accuracy:", accuracy.eval({X: x_test.T, Y: y_test.T, D: 1.0}))
        
train()

### Evaluate the Model

The last three steps above is where our test data was ran through the model and produced how accurate our model was on the test data.

After training the model, we created a node for prediction. This node compares two vectors, our predicted labels and our actual labels. Each vector is 10 elements long with a 1 in the predicted/actual digit location. So we are comparing the vectors. If they match (prediction matches actual), then we have a TRUE; otherwise a FALSE. That's how we are going to get our accuracy percentage calculated.

Next, we create the node accuracy. This node is a cost function!

We then run the accuracy node, feeding it the test images as the X variable and the test labels as the Y variable. This will result in the test images being ran through the model (which is in memory) and the corresponding output vectors evaluated against the actual labels of the test images (Y).

### Rerun with different learning rate

Wah! We only got 3% accuracy with our learning rate at 0.5

It appears we were converging, but more likely wildly bouncing back and forth on reducing the cost of each batch. This might be an infinite gradient problem. Let's try lowering the learning are by a magnitude.

In [ ]:
learning_rate = 0.05
train()

### Rerun with different dropout

Wow, we got 99.9% accuracy! We probably won't do better. The training and test accuracy are nearly the same, so we do not have an overfitting problem. But for the fun of it, let's increase the dropout rate and see what happens.

In [ ]:
keep_prob = 0.5
train()

Not really any difference!

Let's try lowering our batch size by a magnitude. This will cause the optimizer (backward probagation) to run 10 times more often. This will cause it to take longer. Perhaps an increase in accuracy will make it worthwhile?

In [ ]:
batch_size = 20
train()

Well, it took double the time. Our training accuracy whent to 100% - Wow. But the test accuracy is still the same. Looks like the only difference was it took twice as long.

## You have now successful designed and run DNN with Dropout, and feeding hyperparameters.